In [1]:
#=================================================================================================================================================
# keybert : doc에서 키워드등를 추출하는 bert 모델
# - 테스트 해보니, sentencebert 보다는 pretrainBert 모델이 더 낳은 키워드를 출력하는것 같음.
# - sentencebert, bert모델등 임베딩 출력 모델 사용 가능
# - 키워드 추출 시간이 14~18초로 오래 걸림.
#
# 출처 : https://hmkim312.github.io/posts/Keybert%EC%99%80_kiwi%ED%98%95%ED%83%9C%EC%86%8C%EB%B6%84%EC%84%9D%EA%B8%B0%EB%A5%BC_%EC%82%AC%EC%9A%A9%ED%95%98%EC%97%AC_%ED%82%A4%EC%9B%8C%EB%93%9C%EC%B6%94%EC%B6%9C_%ED%95%98%EA%B8%B0/
# 참고 : https://maartengr.github.io/KeyBERT/ 
# 패키지 설치 : pip install keybert
#=================================================================================================================================================
from keybert import KeyBERT
from transformers import AutoModel, BertModel, AlbertModel
from sentence_transformers import models, losses
from sentence_transformers import SentenceTransformer, InputExample

In [3]:
# 모델 로딩
#model_path = "../../data11/model/moco/klue-distil/klue-sbert-v1.0"
model_path = "bongsoo/albert-small-kor-v1"#"klue/bert-base"

'''
# sentencebert 모델일때

#=================================
# sentencebert일때 옵션
do_lower_case = 1
pooling_mode = "mean"
max_seq_length = 128 #128
#=================================
word_embedding_model = models.Transformer(model_path, max_seq_length=max_seq_length, do_lower_case=do_lower_case)

pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),  #모델이 dimension(768)
                              pooling_mode=pooling_mode)  

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
print(model)
'''
# sentencebert 보다는 pretrainmodel 일때 가 더 성능이 좋은것 같음
model = AutoModel.from_pretrained(model_path)
model.eval()
#model = AlbertModel.from_pretrained(model_path)

Some weights of the model checkpoint at bongsoo/albert-small-kor-v1 were not used when initializing AlbertModel: ['predictions.decoder.bias', 'predictions.LayerNorm.weight', 'predictions.dense.weight', 'predictions.dense.bias', 'sop_classifier.classifier.weight', 'predictions.decoder.weight', 'predictions.LayerNorm.bias', 'sop_classifier.classifier.bias', 'predictions.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


AlbertModel(
  (embeddings): AlbertEmbeddings(
    (word_embeddings): Embedding(30000, 128, padding_idx=0)
    (position_embeddings): Embedding(512, 128)
    (token_type_embeddings): Embedding(2, 128)
    (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0, inplace=False)
  )
  (encoder): AlbertTransformer(
    (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
    (albert_layer_groups): ModuleList(
      (0): AlbertLayerGroup(
        (albert_layers): ModuleList(
          (0): AlbertLayer(
            (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (attention): AlbertAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (attention_dropout): Dropout(p=0, inplace=False)
      

In [4]:
text="""
김영하가 『살인자의 기억법』 이후 9 년 만에 내놓는 장편소설 『작별인사』는 그리 멀지 않은 미래를 배경으로, 별안간 삶이 송두리째 뒤흔들린 한 소년의 여정을 좇는다. 
유명한 IT 기업의 연구원인 아버지와 쾌적하고 평화롭게 살아가던 철이는 어느날 갑자기 수용소로 끌려가 난생처음 날것의 감정으로 가득한 혼돈의 세계에 맞닥뜨리게 되면서 정신적, 신체적 위기에 직면한다. 
동시에 자신처럼 사회에서 배제된 자들을 만나 처음으로 생생한 소속감을 느끼고 따뜻한 우정도 싹틔운다. 철이는 그들과 함께 수용소를 탈출하여 집으로 돌아가기 위해 길을 떠나지만 그 여정에는 피할 수 없는 질문이 기다리고 있다.
세계보건기구 WHO가 팬데믹을 선언한 지 2년이 지나서야 작가는 『작별인사』의 개작을 마쳤다. 420매 분량이던 원고는 약 800매로 늘었고, 주제도 완전히 달라졌다. 
‘인간을 인간으로 만드는 것은 무엇인가?’, ‘인간과 인간이 아닌 존재들을 가르는 경계는 어디인가’를 묻던 소설은 ‘삶이란 과연 계속될 가치가 있는 것인가?’, 
‘세상에 만연한 고통을 어떻게 하면 줄일 수 있을 것인가’, ‘어쩔 수 없이 태어났다면 어떻게 살고 어떻게 죽어야 할 것인가’와 같은 질문을 던지는 이야기로 바뀌었다.
팬데믹이 개작에 영향을 주었을 수도 있고, 원래 『작별인사』의 구상에 담긴 어떤 맹아가 오랜 개작을 거치며 발아했는지도 모른다. 그것에 대해 작가는 이렇게 말하고 있다.
마치 제목이 어떤 마력이 있어서 나로 하여금 자기에게 어울리는 이야기로 다시 쓰도록 한 것 같은 느낌이다. 탈고를 하고 얼마 지나지 않아 원고를 다시 읽어보았다. 
이제야 비로소 애초에 내가 쓰려고 했던 어떤 것이 제대로, 남김 없이 다 흘러나왔다는 생각이 들었다. _’작가의 말’에서
전면적인 수정을 통해 2022년의 『작별인사』는 2020년의 『작별인사』를 마치 시놉시스나 초고처럼 보이게 할 정도로 확연하게 달라졌다. 그리고 김영하의 이전 문학 세계와의 연결점들이 분명해졌다.
"""
text = '''
제주도(濟州島, 영어: Jeju-do)는 대한민국의 남해에 위치한 섬이자 대한민국에서 가장 큰 섬이다. 
또한 제주도는 행정구역상 제주특별자치도에 속한다. 섬의 면적은 1833.2 km²[2]인데 이는 남한 면적의 1.83%에 해당한다. 
2020년 기준 주민등록 인구는 약 67만 명으로[1] 대한민국의 섬 중에서 가장 인구가 많다.
또한 제주도는 중앙의 한라산을 중심으로 완만한 경사를 이루어 동서 73 km, 남북 31km의 타원형 모양을 하고 있다. 
일주도로 길이는 181km, 해안선은 258km이다. 제주도의 북쪽 끝은 김녕해수욕장이고, 남쪽 끝은 송악산이며, 서쪽 끝은 수월봉, 동쪽 끝은 성산 일출봉이다. 
제주도는 온대 기후에 속하며, 겨울에도 거의 영하로 떨어지지 않고 영상을 유지한다. 
황해와 동해, 동중국해 경계에 자리잡아 대한민국의 정치·경제는 물론 군사상으로도 중요한 지위에 있다.
또한 제주도는 2002년 12월 16일 유네스코가 기후 및 생물 다양성의 생태계적 가치를 인정하여 제주도를 생물권보전지역을 지정하였다.
가파도 뿐만 아니라 한라산, 성산 일출봉, 거문오름 용암동굴계가 학술·문화·관광·생태 등의 가치와 중요성을 인정받아 2007년 6월, 제주 화산섬과 용암 동굴이라는 이름으로 세계자연유산에 등록되었다. 
2010년 10월 3일 유네스코 세계지질공원네트워크(GGN)가 제주도 지역에 세계지질공원을 인증했다.
'''

In [8]:
import time

starttime = time.time()

kw_model = KeyBERT(model)
keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=5)  # top_n= 키워드 출력 계수
print(keywords)

endtime = time.time() - starttime
print(f'=== 처리시간: {endtime:.3f} 초 ===')


[('대한민국의', 0.577), ('제주도는', 0.5617), ('대한민국에서', 0.561), ('남한', 0.5512), ('제주도', 0.5224)]
=== 처리시간: 13.686 초 ===
